# A quick look into the mass-balance calibration procedure

The default mass-balance (MB) model of OGGM is a very standard [temperature index melt model](https://www.sciencedirect.com/science/article/pii/S0022169403002579). At the beginning, OGGM had a very complicated calibration procedure where the glaciers with in-situ data were calibrated and then a so-called *tstar* got interpolated for glaciers without observations (see the [original publication](https://www.the-cryosphere.net/6/1295/2012/tc-6-1295-2012.html)). This method was very powerful, however, as new observational datasets emerged, we can finally calibrate on a glacier-per-glacier basis. With the new era of geodetic observations, OGGM uses per default the average geodetic observations from Jan 2000--Jan 2020 of [Hugonnet al. 2021](https://www.nature.com/articles/s41586-021-03436-z), that are now available for almost every glacier world-wide. 

In this tutorial, we will:
- [introduce the new default calibration procedure](#Default-calibration-(OGGM->=v1.6)) 
- [show how to calibrate on glaciers with in-situ observations](#Other-calibration-options-for-glaciers-with-additional-observations)
- [show how to deal with errors and how to calibrate on other data](#Dealing-with-errors-and-including-your-own-mass-balance-observations)
- [show the influence of different calibration options and explain the overparameterisation problem](#Overparameteristion-or-the-magic-choice-of-the-best-calibration-option:) (more details are e.g. in [Schuster et al., 2023, in review]())

## Set-up

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import os

import oggm
from oggm import cfg, utils, workflow, tasks, graphics
from oggm.core import massbalance
from oggm.core.massbalance import mb_calibration_from_scalar_mb


In [ ]:
cfg.initialize(logging_level='WARNING')
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-calib-mb', reset=True)
cfg.PARAMS['border'] = 10

We start from two well known glaciers in the Austrian Alps, Kesselwandferner and Hintereisferner. But you can also choose any other other glacier, e.g. from [this list](https://github.com/OGGM/oggm-sample-data/blob/master/wgms/rgi_wgms_links_20220112.csv). 

In [ ]:
# we start from preprocessing level 2
# in OGGM v1.6 you have to explicitly indicate the url from where you want to start from
# we will use here the elevation band flowlines which are much simpler than the centerlines
base_url = ('https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/'
            'L1-L2_files/elev_bands')
gdirs = workflow.init_glacier_directories(['RGI60-11.00787',  'RGI60-11.00897'], from_prepro_level=2,
                                          prepro_base_url=base_url)

We use the preprocessed level 2, so we also need to process the climate ourselves:

In [ ]:
# the climate that OGGM uses per default:
cfg.PARAMS['baseline_climate']

In [ ]:
# creates a climate file with the baseline climate, 
# if you want to change the climate, you have to do that,
# before the calibration!
workflow.execute_entity_task(tasks.process_climate_data, gdirs);

The two glaciers are neighbors but have very different geometries:

In [ ]:
f, ax = plt.subplots(1,1,figsize=(6, 6))
graphics.plot_googlemap(gdirs[:2], ax=ax)
ax.set_title(gdirs[0].rgi_id + ' & ' + gdirs[1].rgi_id);

## Default mass-balance calibration (OGGM >=v1.6)

In [ ]:
cfg.PARAMS['geodetic_mb_period']

Per default the [Hugonnet et al. (2021)](https://www.nature.com/articles/s41586-021-03436-z) average geodetic observation is used over the entire time period Jan 2000 to Jan 2020 to calibrate the melt factor (`melt_f`) for every single glacier. This is done by `mb_calibration_from_scalar_mb` which determines the mass balance parameters from a scalar mass-balance value.

In [ ]:
workflow.execute_entity_task(mb_calibration_from_scalar_mb, gdirs)

In [ ]:
# if you want to know more about the different options and their meanings
# -> check out the docstring:
# mb_calibration_from_scalar_mb?

The output shows you the calibrated MB model parameters, the used observed reference MB for the calibration (`ref_mb`), the time period and the climate source. Between the two glaciers, only the `melt_f` changed. In the default option, the precipitation factor (`prcp_fac`), depends on the average winter precipitation which is the same for the two neighbouring glaciers (they get their climate from the same climate gridpoint). 

The following figure shows the used relationship between winter precipitation and precipitation factor. It was calibrated by adapting `melt_f` and `prcp_fac` to match the average geodetic and winter MB on around 100 glaciers with both informations available. The found relationship of decreasing `prcp_fac` for increasing winter precipitation makes sense, as glaciers with already a large winter precipitation should not be corrected with a large multiplicative `prcp_fac`.  

In [ ]:
w_prcp_array = np.arange(0.5,20,1)
# we basically do here the same as in massbalance.decide_winter_precip_factor(gdir)
a, b = cfg.PARAMS['winter_prcp_fac_ab']
r0, r1 = cfg.PARAMS['winter_prcp_fac_range']
prcp_fac = a * np.log(w_prcp_array) + b
# don't allow extremely low/high prcp. factors!!!
prcp_fac_array = utils.clip_array(prcp_fac, r0, r1)
plt.plot(w_prcp_array, prcp_fac_array)
plt.xlabel(r'winter daily mean precipitation (kg m$^{-2}$ day$^{-1}$)')
plt.ylabel('precipitation factor (prcp_fac)');

In the default calibration option, we don't apply a temperature bias (`temp_bias`), which could potentially correct the climate to the local scale. However, we will change the three MB model parameters (`melt_f`, `temp_bias` and `prcp_fac`) later and see their influence on the calibration. 

There are also some global MB parameters (`mb_global_params`), which we assume to be the same globally. 

In [ ]:
gdirs[-1].read_json('mb_calib')['mb_global_params']

These global MB parameter values were found to represent best the in-situ observations during a cross-validation for glaciers with additional observations. Of course, they could also be changed for different glaciers, but this would need even more data to justify the differences! The influence of using a different temperature lapse rate gradient to the default -0.0065 K/km are analysed in [Schuster et al. (2023, in review)]().

Note that the two glaciers are in the same climate (from the forcing data) but are very different in size, orientation and geometry. So, the resulting `melt_f` and also the MB values are also different:

In [ ]:
for gdir in gdirs:
    mbmod = massbalance.MonthlyTIModel(gdir)
    mean_mb = mbmod.get_specific_mb(fls=gdir.read_pickle('inversion_flowlines'),
                                               year=np.arange(2000,2020,1)).mean()
    print(gdir.rgi_id, f': average MB 2000-2020 = {mean_mb:.1f} kg m-2, ',
          f"melt_f: {gdir.read_json('mb_calib')['melt_f']:.1f} kg m-2 day-1 K-1")

Kesselwandferner has a less negative MB than its neighbor, probably because it is smaller in size and spans less altitude.

*To simplify things, we will now focus just on one glacier (Hintereisferner), but you can repeat it of course with other glaciers:*

Let's check if the calibration worked: 
- We will get first the average modelled MB,

In [ ]:
gdir_hef = gdirs[1]
h, w = gdir_hef.get_inversion_flowline_hw()
mb_geod = massbalance.MonthlyTIModel(gdir_hef)
# Note, if you change cfg.PARAMS['geodetic_mb_period'], you need to change the years here as well!
mbdf= pd.DataFrame(index = np.arange(2000,2020,1))
mbdf['mod_mb'] = mb_geod.get_specific_mb(h, w, year=mbdf.index)
mbdf.mean()

- then get the observed geodetic MB that we calibrated our mass-balance to.

In [ ]:
print('reference MB: ' + str(gdir_hef.read_json('mb_calib')['reference_mb'])+ ' kg m-2')
print('reference MB uncertainties: '+ str(gdir_hef.read_json('mb_calib')['reference_mb_err'])+ ' kg m-2')
print('reference MB time period: ' + gdir_hef.read_json('mb_calib')['reference_period'])
# if you use the default calibration data from Hugonnet et al., 2021, 
# you can also get the geodetic data from any glacier from here:
# ref_geod_mb = utils.get_geodetic_mb_dataframe().loc[gdir_hef.rgi_id]
# ref_geod_mb_period = ref_geod_mb.loc[ref_geod_mb.period==cfg.PARAMS['geodetic_mb_period']].dmdtda*1000

   - We calibrated on the entire 20-yr time period, because then the observational uncertainties are smallest.

In [ ]:
# this tests if the two parameters are very similar:
np.testing.assert_allclose(gdir_hef.read_json('mb_calib')['reference_mb'], mbdf['mod_mb'].mean())

Perfect! Our MB model reproduces the average observed MB, so the calibrated worked!

We have calibrated the `melt_f` to match the average geodetic observation and fixed the other parameters. We will now instead fix the `melt_f` and the `prcp_fac`, and use the `temp_bias` as free variable for calibration (by overwriting the default value of `calibrate_param1`).

In [ ]:
# Let's calibrate on the temp_bias instead
# overwrite_gdir has to be set to True,
# because we want to overwrite the old calibration
mb_calibration_from_scalar_mb(gdir_hef,
                              calibrate_param1='temp_bias',
                              overwrite_gdir=True)

mb_temp_b = massbalance.MonthlyTIModel(gdir_hef)
mbdf['mod_mb_temp_b'] = mb_temp_b.get_specific_mb(h, w, year=mbdf.index)

Let's read the new calibrated MB model parameter for the glacier:

In [ ]:
gdir_hef.read_json('mb_calib')

Here we used the median `melt_f` (i.e., 5 kg m-2 day-1 K-1) and changed `temp_bias` until the `reference MB` is matched. As the `melt_f` is lower than in the previous calibration option, we need to have a positive `temp_bias` to get to the same average MB.

**We can do the same with the precipitation factor (`prcp_fac`):** 

In [ ]:
# Let's calibrate on the prcp_fac instead
# overwrite_gdir has to be set to True,
# because we want to overwrite the old calibration
mb_calibration_from_scalar_mb(gdir_hef,
                              calibrate_param1='prcp_fac',
                              overwrite_gdir=True)

mb_prcp_fac = massbalance.MonthlyTIModel(gdir_hef)
mbdf['mod_mb_prcp_fac'] = mb_prcp_fac.get_specific_mb(h, w, year=mbdf.index)
gdir_hef.read_json('mb_calib')

We chose two glaciers that actually have also in-situ observations available. 
We will get the in-situ observations like that:

In [ ]:
mbdf['in_situ_mb'] = gdir_hef.get_ref_mb_data().loc[2000:2019]['ANNUAL_BALANCE']
# if we want to compare the average to the average geodetic MB, we need to have all years available
assert len(mbdf['in_situ_mb']) == 20

Let's plot how well we match the interannual observations for the different calibration options:

In [ ]:
plt.plot(mbdf['in_situ_mb'], label='in-situ observations\n'+f'average 2000-2020: {mbdf.in_situ_mb.mean():.1f} '+ r'kg m$^{-2}$', color='grey', lw=3)
plt.plot(mbdf['mod_mb'],
         label='modelled mass-balance via calibrating melt_f\n'+f'average 2000-2020: {mbdf.mod_mb.mean():.1f} ' + r'kg m$^{-2}$')
plt.plot(mbdf['mod_mb_temp_b'],
         label='modelled mass-balance via calibrating temp_bias\n'+f'average 2000-2020: {mbdf.mod_mb_temp_b.mean():.1f} ' + r'kg m$^{-2}$')
plt.plot(mbdf['mod_mb_prcp_fac'],
         label='modelled mass-balance via calibrating prcp_fac\n'+f'average 2000-2020: {mbdf.mod_mb_prcp_fac.mean():.1f} ' + r'kg m$^{-2}$')
plt.xticks(np.arange(2000,2020,2))
plt.legend(bbox_to_anchor=(1,1))
plt.ylabel(r'specific mass-balance (kg m$^{-2}$)')
plt.xlabel('Year');

For this glacier, over the same time period (here 2000-2020), the average MB is quite similar between the geodetic and in-situ observation and could even be explained by the fact that the in-situ observations are in hydrological years (starting in October of the previous year) and the geodetic observations are in calendar years. If you repeat the analysis for other glaciers with in-situ observations over that time period, you might see larger discrepancies. 

You can also see, that there are differences in the annual MB between the calibration options and the in-situ observations. We will analyse these differences more systematically later! 

## Other calibration options for glaciers with additional observations

Ok, but actually you might trust the in-situ observations much more than the geodetic observation. So if you are only interested in glaciers with these in-situ observations, you can also use the in-situ observations for calibration. This might allow you also to calibrate over a longer time period and to use additional informations (such as the interannual MB variability, seasonal MB, ...). However, bare in mind that there are often gaps in the in-situ MB time series and make sure that you calibrate to the same modelled time period!

Attention: For the Hintereisferner glacier with an almost 70-year long time series, the assumption that we make, i.e., that the area does not change over the time period, gets more problematic. So think twice before repeating this at home!

In [ ]:
massbalance.mb_calibration_from_wgms_mb(gdir_hef, overwrite_gdir=True)

`mb_calibration_from_wgms_mb` is a short-cut function that uses internally `mb_calibration_from_scalar_mb`,
but uses directly the average annual in-situ MB observations from the WGMS. You can get them like that:

In [ ]:
mbdf_in_situ = gdir_hef.get_ref_mb_data()
mbdf_in_situ['ref_mb'] = mbdf_in_situ['ANNUAL_BALANCE']
# check that every year between the beginning and the end has MB observations 
assert len(mbdf_in_situ.index) == (mbdf_in_situ.index[-1] - mbdf_in_situ.index[0] + 1)
ref_mb = mbdf_in_situ.ref_mb.mean()

In [ ]:
### There exist a short-cut function is to use 
### it does the same thing as above in one line
### (i.e. it calibrates directly on the average in-situ observations

In [ ]:
mb_in_situ_obs = massbalance.MonthlyTIModel(gdir_hef)
mbdf_in_situ['mod_mb'] = mb_in_situ_obs.get_specific_mb(h, w, year=mbdf_in_situ.index)
plt.plot(mbdf_in_situ['ref_mb'],
         label='in-situ observations\n'+f'average: {ref_mb:.1f} '+ r'kg m$^{-2}$', color='grey', lw=3)
plt.plot(mbdf_in_situ['mod_mb'],
         label=('modelled mass-balance\nvia calibrating melt_f\n'
                +f'average: {mbdf_in_situ.mod_mb.mean():.1f} ' + r'kg m$^{-2}$'))

plt.legend()
plt.ylabel(r'specific mass-balance (kg m$^{-2}$)')
plt.xlabel('Year');

The average MB over the entire time period is matched as we calibrate to it. However, the interannual mass-balance variability is different. How well the modelled annual mass-balance time series matches the observations can be for example assessed by looking at the correlation between modelled and observed annual MB 

In [ ]:
mbdf_in_situ[['ref_mb','mod_mb']].corr().values[0][1]

or by looking into the standard deviation of interannual MB variability, where we see a much larger variability for the modelled MB:

In [ ]:
mbdf_in_situ.std()[['ref_mb','mod_mb']]


### Dealing with errors and including your own mass-balance observations

Let's look at another glacier, now in Central Asia, for example the Golubin glacier:

In [ ]:
gdir_2 = workflow.init_glacier_directories(['RGI60-13.11609'], from_prepro_level=2,
                                         prepro_base_url=base_url)[0]

tasks.process_climate_data(gdir_2)

f, ax = plt.subplots(1,1,figsize=(6, 6))
graphics.plot_googlemap([gdir_2], ax=ax)
plt.tight_layout()

Let's calibrate that glacier (with the default average geodetic observation):

In [ ]:
mb_calibration_from_scalar_mb(gdir_2)
# you could also calibrate on the WGMS data instead
# as this glacier has in-situ MB observations
# massbalance.mb_calibration_from_wgms_mb(gdir_2, overwrite_gdir=True)

We got a `RuntimeError` that says that the `ref_mb` is not matched and that we should set `calibrate_param2`. What happened? 

Well, no `melt_f` parameter could be found in the given ranges to reproduce the given calibration data (`ref_mb`). 

In [ ]:
# What are the current minimum and maximum ranges of the melt factor (unit: kg m-2 day-1 K-1)
cfg.PARAMS['melt_f_min'], cfg.PARAMS['melt_f_max']

If we believe that our observations are true, maybe the climate or the processes represented by the MB model are erroneous. 

What can we do to still match the `ref_mb`? We can either change the `melt_f` ranges by setting other values to the parameters above or we can allow that another MB model parameter is changed (i.e., `calibrate_param2`). This is basically very similar to the three-step-calibration 
first introduced in [Huss & Hock 2015](https://doi.org/10.3389/feart.2015.00054), but you can choose your parameter ranges and parameter order yourself. 

**To reduce these MB model calibration errors, we will first change the `melt_f`, fix it at the lower or upper limit, and then change the `temp_bias`. This step-wise calibration is also the option that is used operationally in OGGM>=v1.6 in the preprocessed levels >=3 for all glaciers world-wide!**

In [ ]:
# Allowing another parameter to change is done by defining calibrate_param2
mb_calibration_from_scalar_mb(gdir_2, calibrate_param2='temp_bias')

The `melt_f` is at the minimum value and temperature was corrected to more negative values to allow for the relatively weak negative MB.  

**Use your own or fake MB observations for calibration**

In the same way, you can also use your own MB observations or fake data to calibrate the MB model. We use here as an example, an unrealistically high positive MB for the Hintereisferner over a 10-year time period. To allow the calibration to happen, we need again to set `calibrate_param2`! Here we will use the `prcp_fac` as second parameter for a change:

In [ ]:
ref_period = '2000-01-01_2010-01-01'
ref_mb = 2000 # Let's use an unrealistically positive  mass-balance
mb_calibration_from_scalar_mb(gdir_hef, ref_mb=ref_mb,
                                ref_period=ref_period, overwrite_gdir=True, write_to_gdir=True,
                               calibrate_param2='prcp_fac');


In [ ]:
mb_new = massbalance.MonthlyTIModel(gdir_hef)
# ok, we actually matched the new ref_mb over the 10-yr period
np.testing.assert_allclose(ref_mb,
                           mb_new.get_specific_mb(h, w, year=np.arange(2000,2010,1)).mean())
# Let's look at the calibrated parameters
gdir_hef.read_json('mb_calib')

Ok, in that case, the climate was too warm to allow for such a positive MB. Even the smallest `melt_f` did not allow for such a positive MB. Therefore, the `prcp_fac` was increased to allow that more (solid) precipitation can occur. 

However, also the `prcp_fac` has a limited range: 

In [ ]:
cfg.PARAMS['prcp_fac_min'], cfg.PARAMS['prcp_fac_max']

So, if we increase the `ref_mb` even further, we might even need a third free parameter (i.e., we need to set `calibrate_param3`). 
Let's try it out, by using now the same order as in [Huss & Hock (2015)](https://doi.org/10.3389/feart.2015.00054) (but different allowed parameter changes that also depend on the chosen climate...):

That means, we match the `ref_mb` by 
- first trying to adapt `prcp_fac`,
- then `melt_f`,
- and finally `temp_bias`.

In [ ]:
ref_mb = 3500 # if you replace it with 4000, no combination of parameters can be found for that glacier
mb_calibration_from_scalar_mb(gdir_hef,ref_mb=ref_mb,
                              ref_period=ref_period,
                              calibrate_param1='prcp_fac',
                              calibrate_param2='melt_f',
                              calibrate_param3='temp_bias',
                              overwrite_gdir=True)

In that case, the minimum `melt_f` and the maximum `prcp_fac` are applied. The `temp_bias` is set to a negative value as this results in more solid precipitation. If you increased the `ref_mb` even further, the method will not find any combination as the `temp_bias` also has a limited 
range. If you really want to match the observation, then you would need to change the parameter ranges.

In [ ]:
cfg.PARAMS['temp_bias_min'], cfg.PARAMS['temp_bias_max']

In [ ]:
### check out the docstring for more information about the options
### by uncommenting the line below:
# mb_calibration_from_scalar_mb?

## Overparameteristion or the magic choice of the best calibration option:

We found already some combinations that equally well match the average MB over a given time period. As we only use only one observation per glacier (i.e., per default the average geodetic MB from 2000-2020), but have up to three free MB model parameters, the MB model is overparameterised. That means, there are in theory an infinite amount of calibration options possible that equally well match the one obervation. Let's look a bit more systematically into that:

We will use a range of different `prcp_fac` and then calibrate the `melt_f` accordingly to always match to the default average MB (`ref_mb`) over the reference period (`ref_period`).

In [ ]:
# calibrate the melt_f and annual MB 
pd_prcp_fac_sens = pd.DataFrame(index=np.arange(0.1,5.0,0.3))
spec_mb_prcp_fac_sens_dict = {}
for prcp_fac in pd_prcp_fac_sens.index:
    calib_param = mb_calibration_from_scalar_mb(gdir_hef,
                                                prcp_fac=prcp_fac, overwrite_gdir=True)
    pd_prcp_fac_sens.loc[prcp_fac, 'melt_f'] = calib_param['melt_f']
    mb_prcp_fac_sens = massbalance.MonthlyTIModel(gdir_hef)
    # ok, we actually matched the new ref_mb
    annual_mb = mb_prcp_fac_sens.get_specific_mb(h, w, year=np.arange(2000,2020,1))
    spec_mb_prcp_fac_sens_dict[prcp_fac] = annual_mb
    # let's also check how well the interannual MB variability is matched
    # by comparing the standard deviation of the annual MB to the observed one
    std_quot_annual_mb = annual_mb.std()/mbdf_in_situ.loc[2000:2019, 'ANNUAL_BALANCE'].std()
    pd_prcp_fac_sens.loc[prcp_fac, 'std_quot_annual_mb'] = std_quot_annual_mb

In [ ]:
# let's get some nice colors visualising more precipitation
colors_prcp_fac = plt.get_cmap('viridis_r').colors[10::10]
plt.figure()
for j,prcp_fac in enumerate(pd_prcp_fac_sens.index):
    plt.plot(prcp_fac, pd_prcp_fac_sens.loc[prcp_fac, 'melt_f'],
             'o', color=colors_prcp_fac[j])
plt.ylabel(r'melt_f (kg m$^{-2}$ day$^{-1}$ K$^{-1}$)')
plt.xlabel('prcp_fac');

All these combinations match the average `ref_mb` equally. The larger the chosen `prcp_fac`, the larger needs to be the calibrated `melt_f` when matching to the same average MB.

What is the influence on the chosen parameter combination on other estimates than the average MB?

In [ ]:
plt.figure()
for j,prcp_fac in enumerate(pd_prcp_fac_sens.index):
    plt.plot(np.arange(2000,2020,1),
             spec_mb_prcp_fac_sens_dict[prcp_fac], '-', 
             color=colors_prcp_fac[j], label=f'{prcp_fac:.1f}')
plt.plot(mbdf_in_situ.loc[2000:2019].index,
         mbdf_in_situ.loc[2000:2019]['ANNUAL_BALANCE'],
         color='grey', lw=3, 
        label='observed in-situ')
plt.ylabel(r'Annual mass-balance (kg m$^{-2}$)')
plt.xlabel('Year')
plt.legend(title='prcp_fac:', bbox_to_anchor=(1,1), fontsize=9)
plt.xticks(np.arange(2000,2020,2));
plt.title(gdir_hef.rgi_id);

The larger the `prcp_fac` and the `melt_f`, the larger is the interannual MB variability. For glaciers with in-situ observations, we can find a combination of `prcp_fac` and `melt_f` that has a similar interannnual MB variability than the observations. For example, you can choose a MB model parameter combination where the standard deviation quotient of the annual the modelled and observed MB is near to 1: 

In [ ]:
condi = (pd_prcp_fac_sens['std_quot_annual_mb'] < 1.1) & (pd_prcp_fac_sens['std_quot_annual_mb'] > 0.9)
pd_prcp_fac_sens[condi]

For the Hintereisferner glacier, the best MB model combination to match the interannual MB variability would be for a `prcp_fac` between 1.6 and 1.9 and a `melt_f` between 6 and 6.5 (more in [Schuster et al., 2023, in review]()).

**We can also fix the `prcp_fac` and change the `temp_b` and `melt_f` instead:**

In [ ]:
melt_f_dict_tb = {}
spec_mb_temp_b_sens_dict = {}

for temp_bias in np.arange(-5,5.0,0.5):
    # for too negative temp_bias, no melt_f is found that matches the observations. We would need to 
    # change the prcp_fac , but here we will just look
    # at those combinations where calibration works with a fixed prcp_fac. 
    try:
        calib_param = mb_calibration_from_scalar_mb(gdir_hef, temp_bias=temp_bias, overwrite_gdir=True)
        melt_f_dict_tb[temp_bias] = calib_param['melt_f']
        mb_temp_b_sens = massbalance.MonthlyTIModel(gdir_hef)
        # ok, we actually matched the new ref_mb
        spec_mb_temp_b_sens_dict[temp_bias] = mb_temp_b_sens.get_specific_mb(h, w, year=np.arange(2000,2020,1))
    except RuntimeError: #, 'RGI60-11.00897: ref mb not matched. Try to set calibrate_param2'
        pass


In [ ]:
# let's get a nice colormap centered at temp_bias=0
norm = matplotlib.colors.Normalize(vmin=-5, vmax=5.01)
colors_temp_bias = plt.get_cmap('coolwarm')

plt.figure()
for j,temp_bias in enumerate(melt_f_dict_tb.keys()):
    plt.plot(temp_bias, melt_f_dict_tb[temp_bias], 'o',
             color=colors_temp_bias(norm(temp_bias))) 
plt.ylabel(r'melt_f (kg m$^{-2}$ day$^{-1}$ K$^{-1}$)')
plt.xlabel('temp_bias (°C)');

A lower `melt_f` is needed if a positive `temp_bias` is applied!

In [ ]:
plt.figure()
for temp_bias in melt_f_dict_tb.keys():
    plt.plot(np.arange(2000,2020,1),
             spec_mb_temp_b_sens_dict[temp_bias], '-', 
             color=colors_temp_bias(norm(temp_bias)),
             label=temp_bias)
plt.plot(mbdf_in_situ.loc[2000:2019].index, mbdf_in_situ.loc[2000:2019]['ANNUAL_BALANCE'], color='grey', lw=3, 
        label='observed in-situ')
plt.ylabel(r'Annual mass-balance (kg m$^{-2}$)')
plt.xlabel('Year')
plt.legend(title='temp_bias:', bbox_to_anchor=(1,1))
plt.xticks(np.arange(2000,2020,2));
plt.title(gdir_hef.rgi_id);

And the interannual MB variability gets smaller when large positive `temp_bias` are applied. The parameter combination choice or calibration option also influences the modelled seasonal MB or elevation-dependent MB over the calibration period. Additionally, volume and runoff are influenced by the model parameter choice. If you are further interested in that you can have a look into [Schuster et al (2023, in review)](), which analyses the "Glacier projections sensitivity to temperature-index model choices and calibration strategies". 

Using prior knowledge on the MB model parameters by a Bayesian calibration scheme is a good option to constrain the parameter ranges. Like that you can also directly include the MB observation uncertainties and quantify the parameter uncertainties. You can read about that in [Rounce et al., 2020](https://doi.org/10.1017/jog.2019.91)!

## Take home points

- We illustrated how a mass-balance (MB) model of a glacier can be calibrated in OGGM.  
- We can use different observational data for calibration: 
    - calibrating to geodetic observations using different time periods, to in-situ direct glaciological observations from the WGMS (if available) or to other custom MB data.  
- There exist different ways of calibrating to the average observational data:
    - default is to calibrate the `melt_f`, and having the `prcp_fac` and `temp_bias` fixed. If the calibration does not work, the `temp_bias` is varied aswell. This is the option that is used operationally in OGGM in the preprocessed levels >=3 for all glaciers world-wide.
    - you can also calibrate instead on `prcp_fac` or `temp_bias` and fix the other parameters.
    - However, we showed that the parameter combination choice has an influence on other estimates than the average MB. The model parameter calibration choice can also impact future volume and runoff projections (see [Schuster et al., 2023, in review]()). 
- As user of OGGM, you will most likely just use the default calibration option. However, it is good to be aware of the overparameterisation problem. In addition, if you want to include uncertainties of the MB model calibration, you could include additional experiments that use another calibration option. With more available observational data or improved climate data, you might also be able to use better ways to calibrate the MB model parameters. 


## What's next?
- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)